In [2]:
import pandas
import torch 
import numpy

C:\Users\elabo\AppData\Local\Temp\ipykernel_6008\1957742264.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was too old on your system - pyarrow 10.0.1 is the current minimum supported version as of this release.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas


In [54]:
class TunableAttentionRegression(torch.nn.Module):
    def __init__(self, input_size = 2707, hidden_size = 64,
                output_size = 1, numberOfHeads = 16) -> None:
        super(TunableAttentionRegression, self).__init__()
        self.embedding = torch.nn.Embedding(input_size, 32)
        self.lstm = torch.nn.LSTM(32, hidden_size, batch_first=True)
        self.attention = torch.nn.MultiheadAttention(hidden_size, num_heads=numberOfHeads, batch_first=True) #https://pytorch.org/docs/stable/generated/torch.ao.nn.quantizable.MultiheadAttention.html#multiheadattention
        self.linear1 = torch.nn.Linear(hidden_size*200, hidden_size*100)
        self.linear2 = torch.nn.Linear(hidden_size*100, hidden_size*50)
        self.linear3 = torch.nn.Linear(hidden_size*50, output_size)
        self.relu = torch.nn.ReLU()
        self.sigmoid = torch.nn.Sigmoid()
        
    def forward(self, x) -> torch.Tensor:
        print(x.shape)
        x = x.view(x.size(0), x.size(2)*2)
        print(x.shape)
        embedded = self.embedding(x)
        print(embedded.shape)
        # embedded = embedded.view(*x.size(), -1) #to make it 3D
        lstm_out, _ = self.lstm(embedded)
        print(lstm_out.shape)
        # lstm_out = lstm_out.permute(1, 0, 2)  # [seq_len, batch, hidden_size]
        # print(lstm_out.shape)
        query = lstm_out.permute(0, 1, 2)
        key = lstm_out.permute(0, 1, 2)
        value = lstm_out.permute(0, 1, 2)
        print(query.shape, key.shape, value.shape)
        attention_output, _ = self.attention(query, key, value)
        print(attention_output.shape)
        attention_output_as_2d = attention_output.view(attention_output.size(0), attention_output.size(1)*attention_output.size(2))
        print(attention_output_as_2d.shape)
        output = self.linear1(attention_output_as_2d)
        print(output.shape)
        output = self.relu(output)
        output = self.linear2(output)
        print(output.shape)
        output = self.relu(output)
        output = self.linear3(output)
        print(output.shape)
        output = self.relu(output)
        print(output)
        output = self.sigmoid(output)
        print(output.shape)
        print(output)
        return output
    

sequence = torch.ones(1, 100, dtype=torch.long)
mods = torch.zeros(1, 100, dtype=torch.long)
data = torch.cat((sequence, mods), 0).unsqueeze(0)
model = TunableAttentionRegression()
output = model(data)

torch.Size([1, 2, 100])
torch.Size([1, 200])
torch.Size([1, 200, 32])
torch.Size([1, 200, 64])
torch.Size([1, 200, 64]) torch.Size([1, 200, 64]) torch.Size([1, 200, 64])
torch.Size([1, 200, 64])
torch.Size([1, 12800])
torch.Size([1, 6400])
torch.Size([1, 3200])
torch.Size([1, 1])
tensor([[0.]], grad_fn=<ReluBackward0>)
torch.Size([1, 1])
tensor([[0.5000]], grad_fn=<SigmoidBackward0>)
